<a href="https://colab.research.google.com/github/sundarramamurthy/DS/blob/master/Spark_and_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Py Spark

In [ ]:
!pip install pyspark

# Load required libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from pyspark import SparkContext

try:
    sc = SparkContext('local', 'my_context')
except ValueError:
    print('Spark_Context already exists!')

from pyspark.sql import SparkSession
try:
    spark = SparkSession.builder.appName('pyspark_session_2').getOrCreate()
except ValueError:
    print('Spark_Session already exists!')

Spark_Context already exists!


# Mount your Google Drive to read the CSV file data

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# Load CSV in Pandas

In [ ]:
%time
pandas_dataframe = pd.read_csv("/content/drive/MyDrive/Data/stock_price.csv")
pandas_dataframe.head()

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.34 µs


,Date,Open,High,Low,Close,Volume,ExDividend,SplitRatio,AdjOpen,AdjHigh,AdjLow,AdjClose,AdjVolume,Symbol
0,2018-03-27,173.68,175.15,166.92,168.340,38962839.0,0.0,1.0,173.68,175.15,166.92,168.340,38962839.0,AAPL
1,2018-03-26,168.07,173.10,166.44,172.770,36272617.0,0.0,1.0,168.07,173.10,166.44,172.770,36272617.0,AAPL
2,2018-03-23,168.39,169.92,164.94,164.940,40248954.0,0.0,1.0,168.39,169.92,164.94,164.940,40248954.0,AAPL
3,2018-03-22,170.00,172.68,168.60,168.845,41051076.0,0.0,1.0,170.00,172.68,168.60,168.845,41051076.0,AAPL
4,2018-03-21,175.04,175.09,171.26,171.270,35247358.0,0.0,1.0,175.04,175.09,171.26,171.270,35247358.0,AAPL


# Load Data using Spark

In [ ]:
%time
spark_dataframe = spark.read.csv("/content/drive/MyDrive/Data/stock_price.csv", header=True, inferSchema=True)
print(spark_dataframe.show(5))

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.25 µs
+----------+------+------+------+-------+-----------+----------+----------+-------+-------+------+--------+-----------+------+
|      Date|  Open|  High|   Low|  Close|     Volume|ExDividend|SplitRatio|AdjOpen|AdjHigh|AdjLow|AdjClose|  AdjVolume|Symbol|
+----------+------+------+------+-------+-----------+----------+----------+-------+-------+------+--------+-----------+------+
|2018-03-27|173.68|175.15|166.92| 168.34|3.8962839E7|       0.0|       1.0| 173.68| 175.15|166.92|  168.34|3.8962839E7|  AAPL|
|2018-03-26|168.07| 173.1|166.44| 172.77|3.6272617E7|       0.0|       1.0| 168.07|  173.1|166.44|  172.77|3.6272617E7|  AAPL|
|2018-03-23|168.39|169.92|164.94| 164.94|4.0248954E7|       0.0|       1.0| 168.39| 169.92|164.94|  164.94|4.0248954E7|  AAPL|
|2018-03-22| 170.0|172.68| 168.6|168.845|4.1051076E7|       0.0|       1.0|  170.0| 172.68| 168.6| 168.845|4.1051076E7|  AAPL|
|2018-03-21|175.04|175.09|171.26| 171.27|3.5247

#  Summary Statistics

## In Pandas

In [ ]:
pandas_dataframe.describe()

,Open,High,Low,Close,Volume,ExDividend,SplitRatio,AdjOpen,AdjHigh,AdjLow,AdjClose,AdjVolume
count,5148.000000,5148.000000,5148.000000,5148.000000,5.148000e+03,5148.000000,5148.000000,5148.000000,5148.000000,5148.000000,5148.000000,5.148000e+03
mean,277.859215,280.151335,275.311379,277.804104,2.881661e+07,0.007576,1.001166,194.893137,196.432051,193.218033,194.880789,5.141651e+07
std,287.739104,289.991700,285.233359,287.688769,2.443281e+07,0.120849,0.083624,280.713119,282.844677,278.389845,280.704504,5.150666e+07
min,23.090000,23.320000,22.730000,23.010000,7.900000e+03,0.000000,1.000000,18.937293,19.125928,18.642039,18.871681,7.900000e+03
25%,45.387500,45.695000,45.002600,45.347500,1.146849e+07,0.000000,1.000000,36.907849,37.243491,36.512462,36.924263,2.021791e+07
50%,117.945000,119.180000,116.860000,118.240000,2.477530e+07,0.000000,1.000000,66.676592,67.214047,66.034707,66.607403,3.756888e+07
75%,525.000000,529.674925,519.837500,525.182500,4.129263e+07,0.000000,1.000000,131.540800,132.081385,131.153959,131.563117,6.682210e+07
max,1177.330000,1186.890000,1171.980000,1175.840000,3.193179e+08,3.290000,7.000000,1177.330000,1186.890000,1171.980000,1175.840000,4.702495e+08


## In Spark

In [ ]:
spark_dataframe.describe().show(truncate=True)

+-------+----------+------------------+-----------------+------------------+-----------------+--------------------+--------------------+-------------------+------------------+------------------+------------------+------------------+--------------------+------+
|summary|      Date|              Open|             High|               Low|            Close|              Volume|          ExDividend|         SplitRatio|           AdjOpen|           AdjHigh|            AdjLow|          AdjClose|           AdjVolume|Symbol|
+-------+----------+------------------+-----------------+------------------+-----------------+--------------------+--------------------+-------------------+------------------+------------------+------------------+------------------+--------------------+------+
|  count|      5148|              5148|             5148|              5148|             5148|                5148|                5148|               5148|              5148|              5148|              5148|    

# Correlation among different columns in a DataFrame

## In Pandas

In [ ]:
pandas_dataframe[['Open','Close', 'Volume']].corr()

,Open,Close,Volume
Open,1.000000,0.999862,-0.656712
Close,0.999862,1.000000,-0.656984
Volume,-0.656712,-0.656984,1.000000


## In Spark

In [ ]:
from pyspark.sql.functions import corr
spark_dataframe.select(corr("Open","Volume")).show()

+-------------------+
| corr(Open, Volume)|
+-------------------+
|-0.6567115900695651|
+-------------------+



# Aggregation & Grouping

## In Pandas

In [ ]:
pandas_dataframe.groupby('Symbol')['Open','Close'].min()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,Open,Close
Symbol,,
AAPL,90.00,90.28
GOOG,494.65,492.55
MSFT,23.09,23.01


## In Spark

In [ ]:
from pyspark.sql import functions as F
spark_dataframe.groupBy("Symbol").agg(F.max("Open"),F.max("Close")).show()

+------+---------+----------+
|Symbol|max(Open)|max(Close)|
+------+---------+----------+
|  AAPL|   702.41|     702.1|
|  GOOG|  1177.33|   1175.84|
|  MSFT|     97.0|     96.77|
+------+---------+----------+



# Missing Data Imputation

## In Pandas

Pandas DataFrame provides a fillna() function that can fill missing data items with any string or number. 

Spark DataFrames provide a similar function, but they only allow a value that matches the data type of the corresponding column.

In [ ]:
pandas_dataframe.fillna('NA', inplace = True)
pandas_dataframe.fillna(0, inplace = True)

## In Spark

In [ ]:
spark_dataframe = spark_dataframe.na.fill(0)

# Data Filtering

In [ ]:
pandas_dataframe.loc[pandas_dataframe['Symbol'] == 'AAPL', ['Open','Close']].head()


,Open,Close
0,173.68,168.340
1,168.07,172.770
2,168.39,164.940
3,170.00,168.845
4,175.04,171.270


In [ ]:
spark_dataframe.filter("Symbol == 'GOOG'").select("Open","Close").show(5)

+-------+-------+
|   Open|  Close|
+-------+-------+
| 1063.0| 1005.1|
| 1046.0|1053.21|
|1047.03|1021.57|
|1081.88|1049.08|
|1092.74|1090.88|
+-------+-------+
only showing top 5 rows



# Features Engineering

### Column Selection

In [ ]:
pdf=pandas_dataframe[['Symbol', 'Close']]
sdf=spark_dataframe.select('Symbol', 'Close')

In [ ]:
pdf.head()

,Symbol,Close
0,AAPL,168.340
1,AAPL,172.770
2,AAPL,164.940
3,AAPL,168.845
4,AAPL,171.270


In [ ]:
sdf.show(5)

+------+-------+
|Symbol|  Close|
+------+-------+
|  AAPL| 168.34|
|  AAPL| 172.77|
|  AAPL| 164.94|
|  AAPL|168.845|
|  AAPL| 171.27|
+------+-------+
only showing top 5 rows



### Renaming Columns

In [ ]:
pandas_dataframe.rename(columns = {"Open": "open"}, inplace = True)
spark_dataframe = spark_dataframe.withColumnRenamed("Close", "close")

In [ ]:
pandas_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5148 entries, 0 to 5147
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Date        5148 non-null   object 
 1   open        5148 non-null   float64
 2   High        5148 non-null   float64
 3   Low         5148 non-null   float64
 4   Close       5148 non-null   float64
 5   Volume      5148 non-null   float64
 6   ExDividend  5148 non-null   float64
 7   SplitRatio  5148 non-null   float64
 8   AdjOpen     5148 non-null   float64
 9   AdjHigh     5148 non-null   float64
 10  AdjLow      5148 non-null   float64
 11  AdjClose    5148 non-null   float64
 12  AdjVolume   5148 non-null   float64
 13  Symbol      5148 non-null   object 
dtypes: float64(12), object(2)
memory usage: 563.2+ KB


In [ ]:
spark_dataframe.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = false)
 |-- High: double (nullable = false)
 |-- Low: double (nullable = false)
 |-- close: double (nullable = false)
 |-- Volume: double (nullable = false)
 |-- ExDividend: double (nullable = false)
 |-- SplitRatio: double (nullable = false)
 |-- AdjOpen: double (nullable = false)
 |-- AdjHigh: double (nullable = false)
 |-- AdjLow: double (nullable = false)
 |-- AdjClose: double (nullable = false)
 |-- AdjVolume: double (nullable = false)
 |-- Symbol: string (nullable = true)



### New feature creation

### In Pandas

In [ ]:
pandas_dataframe['diff'] = pandas_dataframe['Close'] - pandas_dataframe['open']


In [ ]:
pandas_dataframe.head()

,Date,open,High,Low,Close,Volume,AdjOpen,AdjHigh,AdjLow,AdjClose,AdjVolume,Symbol,diff,open_scaled
0,2018-03-27,173.68,175.15,166.92,168.340,38962839.0,173.68,175.15,166.92,168.340,38962839.0,AAPL,-5.340,0.130467
1,2018-03-26,168.07,173.10,166.44,172.770,36272617.0,168.07,173.10,166.44,172.770,36272617.0,AAPL,4.700,0.125606
2,2018-03-23,168.39,169.92,164.94,164.940,40248954.0,168.39,169.92,164.94,164.940,40248954.0,AAPL,-3.450,0.125884
3,2018-03-22,170.00,172.68,168.60,168.845,41051076.0,170.00,172.68,168.60,168.845,41051076.0,AAPL,-1.155,0.127279
4,2018-03-21,175.04,175.09,171.26,171.270,35247358.0,175.04,175.09,171.26,171.270,35247358.0,AAPL,-3.770,0.131645


### In Spark

In [ ]:
from pyspark.sql.functions import col
spark_dataframe = spark_dataframe.withColumn("diff", col("close") - col("Open"))

In [ ]:
spark_dataframe.show(5)

+----------+------+------+------+-------+-----------+-------+-------+------+--------+-----------+------+-------------------+-----------+-----------+-----------+
|      Date|  Open|  High|   Low|  close|     Volume|AdjOpen|AdjHigh|AdjLow|AdjClose|  AdjVolume|Symbol|               diff|Symbol_AAPL|Symbol_GOOG|Symbol_MSFT|
+----------+------+------+------+-------+-----------+-------+-------+------+--------+-----------+------+-------------------+-----------+-----------+-----------+
|2018-03-27|173.68|175.15|166.92| 168.34|3.8962839E7| 173.68| 175.15|166.92|  168.34|3.8962839E7|  AAPL| -5.340000000000003|          1|          0|          0|
|2018-03-26|168.07| 173.1|166.44| 172.77|3.6272617E7| 168.07|  173.1|166.44|  172.77|3.6272617E7|  AAPL|  4.700000000000017|          1|          0|          0|
|2018-03-23|168.39|169.92|164.94| 164.94|4.0248954E7| 168.39| 169.92|164.94|  164.94|4.0248954E7|  AAPL|-3.4499999999999886|          1|          0|          0|
|2018-03-22| 170.0|172.68| 168.6|1

# Removing unwanted features

### In Pandas

In [ ]:
pandas_dataframe = pandas_dataframe.drop(['ExDividend', 'SplitRatio'], axis=1)


In [ ]:
pandas_dataframe.head()

,Date,open,High,Low,Close,Volume,AdjOpen,AdjHigh,AdjLow,AdjClose,AdjVolume,Symbol,diff
0,2018-03-27,173.68,175.15,166.92,168.340,38962839.0,173.68,175.15,166.92,168.340,38962839.0,AAPL,342.020
1,2018-03-26,168.07,173.10,166.44,172.770,36272617.0,168.07,173.10,166.44,172.770,36272617.0,AAPL,340.840
2,2018-03-23,168.39,169.92,164.94,164.940,40248954.0,168.39,169.92,164.94,164.940,40248954.0,AAPL,333.330
3,2018-03-22,170.00,172.68,168.60,168.845,41051076.0,170.00,172.68,168.60,168.845,41051076.0,AAPL,338.845
4,2018-03-21,175.04,175.09,171.26,171.270,35247358.0,175.04,175.09,171.26,171.270,35247358.0,AAPL,346.310


### In Spark

In [ ]:
spark_dataframe = spark_dataframe.drop('ExDividend', 'SplitRatio')

In [ ]:
spark_dataframe.show(5)

+----------+------+------+------+-------+-----------+-------+-------+------+--------+-----------+------+------------------+
|      Date|  Open|  High|   Low|  close|     Volume|AdjOpen|AdjHigh|AdjLow|AdjClose|  AdjVolume|Symbol|              diff|
+----------+------+------+------+-------+-----------+-------+-------+------+--------+-----------+------+------------------+
|2018-03-27|173.68|175.15|166.92| 168.34|3.8962839E7| 173.68| 175.15|166.92|  168.34|3.8962839E7|  AAPL|            342.02|
|2018-03-26|168.07| 173.1|166.44| 172.77|3.6272617E7| 168.07|  173.1|166.44|  172.77|3.6272617E7|  AAPL|340.84000000000003|
|2018-03-23|168.39|169.92|164.94| 164.94|4.0248954E7| 168.39| 169.92|164.94|  164.94|4.0248954E7|  AAPL|            333.33|
|2018-03-22| 170.0|172.68| 168.6|168.845|4.1051076E7|  170.0| 172.68| 168.6| 168.845|4.1051076E7|  AAPL|           338.845|
|2018-03-21|175.04|175.09|171.26| 171.27|3.5247358E7| 175.04| 175.09|171.26|  171.27|3.5247358E7|  AAPL|            346.31|
+-------

### Feature Scaling

### In Pandas

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()

In [ ]:
pandas_dataframe['open_scaled'] = scaler.fit_transform(pandas_dataframe['open'].values.reshape(-1,1))

In [ ]:
pandas_dataframe.head()

,Date,open,High,Low,Close,Volume,AdjOpen,AdjHigh,AdjLow,AdjClose,AdjVolume,Symbol,diff,open_scaled
0,2018-03-27,173.68,175.15,166.92,168.340,38962839.0,173.68,175.15,166.92,168.340,38962839.0,AAPL,342.020,0.130467
1,2018-03-26,168.07,173.10,166.44,172.770,36272617.0,168.07,173.10,166.44,172.770,36272617.0,AAPL,340.840,0.125606
2,2018-03-23,168.39,169.92,164.94,164.940,40248954.0,168.39,169.92,164.94,164.940,40248954.0,AAPL,333.330,0.125884
3,2018-03-22,170.00,172.68,168.60,168.845,41051076.0,170.00,172.68,168.60,168.845,41051076.0,AAPL,338.845,0.127279
4,2018-03-21,175.04,175.09,171.26,171.270,35247358.0,175.04,175.09,171.26,171.270,35247358.0,AAPL,346.310,0.131645


In [ ]:
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler



In [ ]:
# VectorAssembler Transformation - Converting column to vector type
assembler = VectorAssembler(inputCols=['Open'], outputCol="open_vec")
scaler = MinMaxScaler(inputCol="open_vec", outputCol="open_scaled")
# Pipeline of VectorAssembler and MinMaxScaler
pipeline = Pipeline(stages=[assembler, scaler])
# Fitting pipeline on dataframe
spark_df_scaled = pipeline.fit(spark_dataframe).transform(spark_dataframe).drop("open_vec")

In [ ]:
spark_df_scaled.show(5)

+----------+------+------+------+-------+-----------+-------+-------+------+--------+-----------+------+------------------+--------------------+
|      Date|  Open|  High|   Low|  close|     Volume|AdjOpen|AdjHigh|AdjLow|AdjClose|  AdjVolume|Symbol|              diff|         open_scaled|
+----------+------+------+------+-------+-----------+-------+-------+------+--------+-----------+------+------------------+--------------------+
|2018-03-27|173.68|175.15|166.92| 168.34|3.8962839E7| 173.68| 175.15|166.92|  168.34|3.8962839E7|  AAPL|            342.02|[0.1304668006653729]|
|2018-03-26|168.07| 173.1|166.44| 172.77|3.6272617E7| 168.07|  173.1|166.44|  172.77|3.6272617E7|  AAPL|340.84000000000003|[0.12560645966176...|
|2018-03-23|168.39|169.92|164.94| 164.94|4.0248954E7| 168.39| 169.92|164.94|  164.94|4.0248954E7|  AAPL|            333.33|[0.12588369836429...|
|2018-03-22| 170.0|172.68| 168.6|168.845|4.1051076E7|  170.0| 172.68| 168.6| 168.845|4.1051076E7|  AAPL|           338.845|[0.1272

#One Hot Encoding

###In Pandas

In [ ]:
pandas_dataframe_ohe = pd.get_dummies(pandas_dataframe)

In [ ]:
pandas_dataframe_ohe.head()

,open,High,Low,Close,Volume,AdjOpen,AdjHigh,AdjLow,AdjClose,AdjVolume,diff,open_scaled,Date_2010-01-04,Date_2010-01-05,Date_2010-01-06,Date_2010-01-07,Date_2010-01-08,Date_2010-01-11,Date_2010-01-12,Date_2010-01-13,Date_2010-01-14,Date_2010-01-15,Date_2010-01-19,Date_2010-01-20,Date_2010-01-21,Date_2010-01-22,Date_2010-01-25,Date_2010-01-26,Date_2010-01-27,Date_2010-01-28,Date_2010-01-29,Date_2010-02-01,Date_2010-02-02,Date_2010-02-03,Date_2010-02-04,Date_2010-02-05,Date_2010-02-08,Date_2010-02-09,Date_2010-02-10,Date_2010-02-11,...,Date_2018-02-02,Date_2018-02-05,Date_2018-02-06,Date_2018-02-07,Date_2018-02-08,Date_2018-02-09,Date_2018-02-12,Date_2018-02-13,Date_2018-02-14,Date_2018-02-15,Date_2018-02-16,Date_2018-02-20,Date_2018-02-21,Date_2018-02-22,Date_2018-02-23,Date_2018-02-26,Date_2018-02-27,Date_2018-02-28,Date_2018-03-01,Date_2018-03-02,Date_2018-03-05,Date_2018-03-06,Date_2018-03-07,Date_2018-03-08,Date_2018-03-09,Date_2018-03-12,Date_2018-03-13,Date_2018-03-14,Date_2018-03-15,Date_2018-03-16,Date_2018-03-19,Date_2018-03-20,Date_2018-03-21,Date_2018-03-22,Date_2018-03-23,Date_2018-03-26,Date_2018-03-27,Symbol_AAPL,Symbol_GOOG,Symbol_MSFT
0,173.68,175.15,166.92,168.340,38962839.0,173.68,175.15,166.92,168.340,38962839.0,342.020,0.130467,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0
1,168.07,173.10,166.44,172.770,36272617.0,168.07,173.10,166.44,172.770,36272617.0,340.840,0.125606,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
2,168.39,169.92,164.94,164.940,40248954.0,168.39,169.92,164.94,164.940,40248954.0,333.330,0.125884,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
3,170.00,172.68,168.60,168.845,41051076.0,170.00,172.68,168.60,168.845,41051076.0,338.845,0.127279,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
4,175.04,175.09,171.26,171.270,35247358.0,175.04,175.09,171.26,171.270,35247358.0,346.310,0.131645,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0


###In Spark


In [ ]:
from pyspark.sql.functions import udf,col
from pyspark.sql.types import IntegerType

categories = spark_dataframe.select('Symbol').distinct().rdd.flatMap(lambda x : x).collect()

categories.sort()

for category in categories:
    function = udf(lambda item: 1 if item == category else 0, IntegerType())
    new_column_name = 'Symbol'+'_'+category
    spark_dataframe = spark_dataframe.withColumn(new_column_name, function(col('Symbol')))

In [ ]:
spark_dataframe.show()

+----------+------+-------+--------+-------+-----------+-------+-------+--------+--------+-----------+------+------------------+-----------+-----------+-----------+
|      Date|  Open|   High|     Low|  close|     Volume|AdjOpen|AdjHigh|  AdjLow|AdjClose|  AdjVolume|Symbol|              diff|Symbol_AAPL|Symbol_GOOG|Symbol_MSFT|
+----------+------+-------+--------+-------+-----------+-------+-------+--------+--------+-----------+------+------------------+-----------+-----------+-----------+
|2018-03-27|173.68| 175.15|  166.92| 168.34|3.8962839E7| 173.68| 175.15|  166.92|  168.34|3.8962839E7|  AAPL|            342.02|          1|          0|          0|
|2018-03-26|168.07|  173.1|  166.44| 172.77|3.6272617E7| 168.07|  173.1|  166.44|  172.77|3.6272617E7|  AAPL|340.84000000000003|          1|          0|          0|
|2018-03-23|168.39| 169.92|  164.94| 164.94|4.0248954E7| 168.39| 169.92|  164.94|  164.94|4.0248954E7|  AAPL|            333.33|          1|          0|          0|
|2018-03-2